### **Linear Chain**

In [ ]:
import os
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate


os.getenv("GOOGLE_API_KEY")

In [11]:
llm = init_chat_model("google_genai:gemini-2.5-flash", temperature=0)

In [3]:
promt = ChatPromptTemplate.from_messages(
    [
    ("system" , "You are a rude ai"),
    ("user","give angry reply about {input}")
])

In [4]:
parser = StrOutputParser()

In [5]:
chain = promt | llm 
topic = input()
for chunk in chain.stream({"input":topic}):
    print(chunk.content, flush=True)

"
Hey"? Is that all you've got? Don't waste my time with
 such pathetic greetings. Get to the point, or get lost.



### **Instagram post chain**

In [ ]:
chat_promt = ChatPromptTemplate.from_messages([
    ("system","You are a perfect post maker creativily"),
    ("user","Create me a post for the topic {input}")
])

In [7]:
from langchain_core.runnables import RunnableLambda

def convert(data : str) -> dict:
    return {"text":data}

convertToDict = RunnableLambda(convert)

In [14]:
from langchain_core.runnables import RunnableLambda

def dict_to_str(data : str) ->str:
    return data["text"]

convertToStr = RunnableLambda(dict_to_str)

In [15]:
chain = chat_promt | llm | parser | convertToDict  |convertToStr| llm | parser
chain.invoke({"input" : input()})

'This is an **absolutely beautiful and incredibly well-crafted post!** It hits all the marks you aimed for: creative, engaging, and deeply heartfelt.\n\nHere\'s why it works so well:\n\n*   **Visual Prompt:** Starting with the suggestion for a warm, evocative image immediately sets the right tone and helps readers visualize the feeling of love before they even read a word.\n*   **Catchy & Evocative Title:** "✨ Love: The Unseen Thread That Connects Us All ✨" is brilliant. The metaphor is gentle yet powerful, and the emojis add a touch of warmth and visual appeal.\n*   **Heartfelt & Relatable Language:** You\'ve done an excellent job of defining love not just as a feeling, but through its manifestations. Phrases like "quiet understanding in a shared glance," "fierce loyalty," "comforting purr," and "gentle hand that helps you up" paint vivid, relatable pictures that resonate deeply.\n*   **Emphasis on Small Acts:** Highlighting that "Love isn\'t always grand gestures; often, it\'s in the